
# Creating and Running a Docker Container from the Shell

This notebook cell documents how to build and run a **minimal Docker container locally**, using a Python script as payload and writing results back to the host system.

---

## Overview of Files

After completing the steps below, the working directory contains:

```bash
02_code_to_execute_in_docker.py
02_dockerfile.txt
02_docker_build_output.txt
02_docker_run_output.txt
02_docker_output.txt
```

Purpose of each file:

* **02_code_to_execute_in_docker.py**
  Python code executed *inside* the container.

* **02_dockerfile.txt**
  Defines the container runtime (OS + Python + execution command).

* **02_docker_build_output.txt**
  Captured output of the `docker build` command.

* **02_docker_run_output.txt**
  Captured output of the `docker run` command.

* **02_docker_output.txt**
  File written by the container and exposed to the host via a volume mount.

---

## Step 1 — Python Code Executed in Docker

Create the file `02_code_to_execute_in_docker.py`:

```python
from pathlib import Path

print("Hello from Docker")

out = Path("02_docker_output.txt")
out.write_text("This file was generated inside a Docker container.\n")

print("Wrote:", out)
```

Notes:

* This is **plain Python**, no Docker-specific code.
* The output file is written **relative to the working directory**.

---

## Step 2 — Dockerfile (Runtime Definition)

Create the Dockerfile as `02_dockerfile.txt`:

```dockerfile
FROM python:3.11-slim

WORKDIR /app

COPY 02_code_to_execute_in_docker.py /app/

CMD ["python", "02_code_to_execute_in_docker.py"]
```

Important points:

* `FROM python:3.11-slim` fixes the Python version and OS layer.
* The Docker image already acts as a **virtual environment**.
* No `venv` activation is required.

---

## Step 3 — Build the Docker Image (Shell)

From the directory containing the files above, run:

```bash
docker build -t python-hello-docker -f 02_dockerfile.txt .
```

To capture the build output:

```bash
docker build -t python-hello-docker -f 02_dockerfile.txt . \
  | tee 02_docker_build_output.txt
```

Result:

* A Docker image named `python-hello-docker` is created.
* The build log is stored for documentation and debugging.

---

## Step 4 — Run the Container (Shell)

Execute the container and expose the working directory:

```bash
docker run --rm \
  -v "$(pwd):/app" \
  python-hello-docker
```

To capture the runtime output:

```bash
docker run --rm \
  -v "$(pwd):/app" \
  python-hello-docker \
  | tee 02_docker_run_output.txt
```

Explanation:

* `--rm` removes the container after execution.
* `-v "$(pwd):/app"` mounts the current directory into the container.
* Files written inside `/app` appear on the host.

---

## Step 5 — Verify Host-Side Output

After execution, verify that the container produced output:

```bash
ls 02_docker_output.txt
cat 02_docker_output.txt
```

This confirms:

* The container ran successfully.
* Data was written **from inside Docker to the host filesystem**.

---

## Conceptual Summary

* The **Dockerfile freezes the runtime** (OS + Python + libraries).
* The **Python code remains unchanged**.
* The **Docker image already provides isolation**, similar to a virtual environment.
* **Volume mounts control where results appear**.

This is the **atomic execution unit** used later in CI/CD and MLOps pipelines.

```

If you want next:
- the **GitHub Actions version** in the same Markdown style, or  
- a **student exercise variant** with TODO markers.

